## TODO

- figure out how to import one notebook into another!
    - https://jupyter-notebook.readthedocs.io/en/stable/examples/Notebook/Importing%20Notebooks.html
- load multiple txt files in to create time-to-test-result graphs


In [1]:
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
%config InlineBackend.print_figure_kwargs = {'bbox_inches':None}
import seaborn as sns
import datetime
import numpy as np
import os
from pathlib import Path, PurePath
from tqdm import tqdm
from collections import defaultdict
import re
import pathlib

In [2]:
#plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode
init_notebook_mode(connected = True)

## Setup

In [55]:
base_loc = '.'
chesco_loc = f'{base_loc}/chesco/data'
chesco_path = pathlib.Path(chesco)
population = 566747

## Functions for loading data

In [56]:
start_date = datetime.datetime(2020, 3, 1, 12, 0, 0)
txtfiles = sorted([path for path in chesco_path.iterdir() if path.name.endswith('.txt')])

In [66]:
df = pd.read_csv(txtfiles[-1], sep='\t', names=['Confirmed', 'Negative', 'Deaths'])
dates = [(start_date + datetime.timedelta(days=i)) for i in range(len(df))]
df['Last_Update'] = dates
df = df[['Last_Update', 'Confirmed', 'Negative', 'Deaths']]

In [67]:
new_cases = []
previous = 0
for i, row in df.iterrows():
    delta = row.Confirmed - previous
    new_cases.append(delta)
    previous = row.Confirmed
df['New_Cases'] = new_cases

df['Population'] = [population] * len(df)

In [68]:
df

,Last_Update,Confirmed,Negative,Deaths,New_Cases,Population
0,2020-03-01 12:00:00,0,0,0,0,566747
1,2020-03-02 12:00:00,0,0,0,0,566747
2,2020-03-03 12:00:00,0,0,0,0,566747
3,2020-03-04 12:00:00,0,0,0,0,566747
4,2020-03-05 12:00:00,1,0,0,1,566747
...,...,...,...,...,...,...
112,2020-06-21 12:00:00,7140,33397,691,11,566747
113,2020-06-22 12:00:00,7152,34092,692,12,566747
114,2020-06-23 12:00:00,7161,34400,693,9,566747
115,2020-06-24 12:00:00,7164,34474,693,3,566747


In [72]:
# Copied from plots

plt_color = {'blue': '#1f77b4', 'orange': '#ff7f0e', 'green': '#2ca02c', 'purple': '#9467bd', 'olive': '#bcbd22'}

def yellow_target_plotly(df, label, output=None, pngScale=None):
    population = set(df.Population).pop()

    days = 14
    target = 50

    percap = df[f'day_avg_{days}']*100000 / population * 14
    target_lst = [target] * len(df.Last_Update)

    fig = go.Figure()
    fig.add_trace(
            go.Scatter(
                x = df.Last_Update,
                y = percap,
                mode = 'lines+markers',
                name = f'{days} day per 100K',
                line_color = plt_color['blue'],
                hovertemplate = '<b>%{y:.1f}</b>',
            )
    )
    fig.add_trace(
        go.Scatter(
            x = df.Last_Update,
            y = target_lst,
            mode = 'lines',
            name = f'{target} cases per 100K',
            line_color = plt_color['orange'],
            hoverinfo = "none",
        )
    )

    layout = go.Layout(
        showlegend=False,  # updated for inline and html
        xaxis_title="Date",
        yaxis_title=f"Positive tests",
        font=dict(
            size=12,
            color="#7f7f7f"
        ),
        hovermode="x unified",
    )
    fig.update_layout(layout)
    title = f"New cases over 14 days per 100K residents: {label}"
    write_figure_plotly(fig, output, title, 'yellow_target', pngScale=pngScale)


In [73]:
# Copied from plots
def average_new_cases(df, days, centered=False):
    """ this computes day the trailing average in the final day """
    """ compute the moving average over {days} days and add as day_avg_{days} to the df """
    field = f'day_avg_{days}'
    df[field] = df.New_Cases.rolling(window=14, min_periods=1, center=centered).mean()

In [75]:
# Copied from plots
def write_figure_plotly(fig, output, title, description, pngScale=None):
    full_layout = go.Layout(
        title=title,
        showlegend=True,
    )
    if output == 'inline':
        fig.update_layout(full_layout)
        fig.show()
    else:
        output = output.replace("'","").replace('.png', f'_{description}.png')
        fig.write_image(output, scale=pngScale)
        fig.update_layout(full_layout)
        output = output.replace('.png', '.html')
        #fig.write_html(output, include_plotlyjs="plotly.min.js", full_html=True)
        fig.write_html(output, include_plotlyjs=False, full_html=False)

In [78]:
average_new_cases(df, 14, centered=False)
yellow_target_plotly(df, "Delaware County (via CC Health Dept)", output='inline', pngScale=1)
